In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
import importlib
import mr_rrn, data_aux

In [ ]:
# Settings
examples=50000
num_steps = 20
num_seq = 3
n_hidden_enc = 4
n_hidden_con = 5
n_hidden_dec = 8
batch_size = 100
learning_rate = 0.01
embedding_shape = [3, 3] # [voc_size, emb_dim]

In [ ]:
importlib.reload(mr_rrn)
# Build graph
timer = datetime.now()
tf.reset_default_graph()
(vocab_input, sequence_input, total_loss, train_step) = mr_rrn.build_graph(embedding_shape, num_seq, num_steps, batch_size, n_hidden_enc, n_hidden_con, n_hidden_dec, learning_rate)
timer = datetime.now() - timer
print('Time build graph: ', timer)

# Load data
X = data_aux.gen_data_sequence(examples=examples, num_seq=num_seq, num_steps=num_steps)
batches = data_aux.batch_iter(list(X), batch_size = batch_size, num_epochs = 3)

# Train
W_embedding = data_aux.random_embedding(embedding_shape[0], embedding_shape[1])
acc_loss = 0
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables(), feed_dict={vocab_input : W_embedding})
    timer = datetime.now()
    for i, batch in enumerate(batches):
        x_batch = np.array(batch)
        feed_dict = {sequence_input : x_batch,
                     }
        loss, _ = sess.run([total_loss, train_step], feed_dict=feed_dict)
        acc_loss += loss
        if i % 25 == 0 and i > 0:
            print(acc_loss/25)
            acc_loss = 0
timer = datetime.now() - timer
print('Time run graph: ', timer)